In [1]:
import numpy as np
from cellpose import models, core, io, plot
from pathlib import Path
from tqdm import trange
from natsort import natsorted


io.logger_setup() # run this to get printing of progress

#Check if colab notebook instance has GPU access

model = models.CellposeModel(gpu=True)

creating new log file
2025-06-25 10:08:55,884 [INFO] WRITING LOG OUTPUT TO C:\Users\75695\.cellpose\run.log
2025-06-25 10:08:55,885 [INFO] 
cellpose version: 	3.1.1.2 
platform:       	win32 
python version: 	3.10.18 
torch version:  	2.7.1+cpu
2025-06-25 10:08:55,908 [INFO] Neither TORCH CUDA nor MPS version not installed/working.
2025-06-25 10:08:55,909 [INFO] >>>> using CPU
2025-06-25 10:08:55,910 [INFO] >>>> using CPU
2025-06-25 10:08:55,984 [INFO] >>>> no model weights loaded


In [3]:
# *** change to your google drive folder path ***
dir = r"C:\Users\75695\Desktop\chot"
dir = Path(dir)
if not dir.exists():
  raise FileNotFoundError("directory does not exist")

# *** change to your image extension ***
image_ext = ".tif"

# list all files
files = natsorted([f for f in dir.glob("*"+image_ext) if "_seg" not in f.name and "_flows" not in f.name])

if(len(files)==0):
  raise FileNotFoundError("no image files found, did you specify the correct folder and extension?")
else:
  print(f"{len(files)} images in folder:")

for f in files:
  print(f.name)

2 images in folder:
t000001xy1c1.tif
t000001xy7c1.tif


In [4]:
# 假设 img 是 (H, W) 形式的灰度图
flow_threshold = 0.4
cellprob_threshold = 0.0
tile_norm_blocksize = 0


In [5]:
masks_ext = ".png" if image_ext == ".png" else ".tif"

for i in trange(len(files)):
    f = files[i]
    img = io.imread(f)
    masks, flows, styles = model.eval(img, batch_size=32, flow_threshold=flow_threshold, cellprob_threshold=cellprob_threshold,
                                  normalize={"tile_norm_blocksize": tile_norm_blocksize})
    masks_uint16 = masks.astype(np.uint16)
    # 供 GUI 使用的掩膜文件
    mask_img_path = f.parent / (f.stem + "_masks" + masks_ext)
    io.imsave(mask_img_path, masks_uint16)
    # GUI 读取的 .npy 掩膜数据（仿 GUI 保存格式）
    mask_npy_path = f.parent / (f.stem + "_seg.npy")
    mask_data = {
        'masks': masks.astype(np.uint16),
        'outlines': None,
        'chan_choose': None,
        'ismanual': False
    }
    np.save(mask_npy_path, mask_data)

100%|██████████| 2/2 [00:02<00:00,  1.10s/it]
